# **Project Title:** Effects of Various Memory Infrastructure

# **Conceptual Question**

In this project, I will be exploring the RNN models. From the class I remembered professor teaching us about RNN's and telling how the hidden state is passed to the next hiddden layer to predict the next token. I understood the reason - to give the model more context about what token to predict - but I could not understand why passing this hidden layer is different than passing the previously predicted tokens directly like in n-gram nlp prediction.

So I setup this experiment to find what is in the hidden state which is being passed in RNN and how the model prediction would change if I just pass the previous 'n' tokens? How effective are the other memory support infrastructures?


# **Hypothesis**

For simple short-range sequence prediction tasks, a standard RNN behaves similarly to a n-gram version of RNN. Both rely primarily on the most recent token, so providing the last token as explicit input can effectively compensate for removing the hidden state.

As the task's required dependency range increases, the RNN's hidden state becomes increasingly advantageous compared to the n-gram model, because the hidden state can carry information from far earlier in the sequence in a fixed-size vector, while an n-gram model must explicitly expand its context window to match the performance.

LSTM model, RNN with attention, and Bidirectional RNN would prove to be better than Vanilla RNN and N-gram model in both the tasks.

# **Experimental Design**

I will be performing this experiment over 2 tasks.

Tasks

We construct sequence prediction tasks where the model predicts a final token, which is a copy of an earlier token. This allows us to control and gradually increase the dependency range. The input is an incomplete palindromic sequence and the output should only be next token in the sequence.
The length of the input varies between {5, 7, 9, 11, 13} and the forward part of the sequence is always more than half the length of the sequence, for example if total length is 5 then the sequence cannot be ABCBA (forward length = 3, backward length = 2). Here is a total list of sequence length and their corresponding allowed forward lengths:
5: [4], 7: [4, 5], 9: [5, 6, 7], 11: [6, 7, 8, 9], 13: [7, 8, 9, 10, 11]. Each combination has equal weightage in the dataset.


1. Task 1:
The sequence can be started from any character but needs to follow alphabetical order. For ease, the letters are cyclical:
A -> B -> C -> … -> Y -> Z -> A -> B -> …

Examples:

A B C D E D C → predict B

F G H I H → predict G

X Y Z A Z → predict Y

K L M N O P Q → predict P


3.  Task 2:
The sequence can be started from any character and does not need to follow any alphabetical order. Forcing the model to remember the sequence instead of using the last token to predict the output.

Examples:

A J X D R D X → predict J

F A C I C → predict A

X S Z A Z → predict S

K T A J I E P → predict E

# **Experimental Code**

In [ ]:
import random
import string
from typing import List, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import math

# Creating Datasets

In [ ]:
ALPHABET = list(string.ascii_uppercase)

def generate_single_sequence(task, seq_len, forward_len, generated, offset=0) -> Tuple[List[str], str]:
    alphabet = list(ALPHABET)

    if task==2:
      random.shuffle(alphabet)

    start_idx = random.randint(0, len(alphabet) - 1) if task==2 else offset
    seq = [alphabet[(start_idx + i) % len(alphabet)] for i in range(1, forward_len+1)]
    seq += [alphabet[(start_idx + (forward_len - (i-forward_len))) % len(alphabet)] for i in range(forward_len+1, seq_len+1)]

    target_token = seq[-1]
    input_sequence = seq[:-1]

    if (input_sequence, target_token) in generated: #avoid dataset pollution
      return generate_single_sequence(task, seq_len, forward_len, generated, offset+1)

    return input_sequence, target_token


def generate_dataset(num_sequences, task, train_ratio = 0.8, seed = 42):
    random.seed(seed)


    lengths = {5: [4], 7: [4, 5], 9: [5, 6, 7], 11: [6, 7, 8, 9], 13: [7, 8, 9, 10, 11]}
    train = []
    test = []

    if task==1:
      for i in lengths:
        for j in lengths[i]:
          dataset = []
          for k in range(26):
            seq, target = generate_single_sequence(task, i, j, dataset, k)
            dataset.append((seq, target))
          split_idx = int(len(dataset) * train_ratio)
          random.shuffle(dataset)
          train += dataset[:split_idx]
          test += dataset[split_idx:]
    else:
      for i in lengths:
        per_sublength = int((0.2*num_sequences)//len(lengths[i]))
        for j in lengths[i]:
          dataset = []
          for k in range(per_sublength):
            seq, target = generate_single_sequence(task, i, j, dataset)
            dataset.append((seq, target))
          split_idx = int(len(dataset) * train_ratio)
          random.shuffle(dataset)
          train += dataset[:split_idx]
          test += dataset[split_idx:]

    random.shuffle(train)
    random.shuffle(test)
    return train, test

# RNN model

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size, num_layers=1):
    super().__init__()
    self.rnn = nn.RNN(
          input_size=input_size,
          hidden_size=hidden_size,
          num_layers=num_layers,
          batch_first=True
        )
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    output, h_last = self.rnn(x)
    logits = self.fc(output[:, -1, :])
    return logits

# N-gram Model

In [ ]:
class Ngram(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size, n):
        super().__init__()
        self.n = n
        self.fc1 = nn.Linear(input_size * (n-1), hidden_size)
        self.fc2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        batch_size, seq_len, in_dim = x.shape
        if seq_len < self.n:
            pad_len = self.n - seq_len
            pad = x.new_zeros(batch_size, pad_len, in_dim)
            x_padded = torch.cat([pad, x], dim=1)
            window = x_padded[:, -self.n+1:, :]
        else:
            window = x[:, -self.n+1:, :]

        window_flat = window.reshape(batch_size, -1)

        h = torch.tanh(self.fc1(window_flat))
        logits = self.fc2(h)

        return logits

# LSTM Model

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size, num_layers=2):
    super().__init__()
    self.lstm = nn.LSTM(
          input_size=input_size,
          hidden_size=hidden_size,
          num_layers=num_layers,
          batch_first=True
        )
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    output, (h_last, c_last) = self.lstm(x)
    logits = self.fc(output[:, -1, :])
    return logits

# RNN with attention

In [ ]:
class RNNAttention(nn.Module):
    def __init__(self, input_size, hidden_size, attn_dim, vocab_size):
        super().__init__()

        self.Wq = nn.Linear(input_size, attn_dim)
        self.Wk = nn.Linear(input_size, attn_dim)
        self.Wv = nn.Linear(input_size, attn_dim)

        self.rnn_cell = nn.RNNCell(input_size + attn_dim, hidden_size)

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, X):
        X = X.squeeze(0)
        total_length = X.shape[0]

        h = X.new_zeros(self.rnn_cell.hidden_size)

        K = self.Wk(X)
        V = self.Wv(X)

        for i in range(total_length):
            q = self.Wq(X[i])
            r = q @ K.T
            a = F.softmax(r, dim = 0)
            c = a @ V
            inp = torch.cat([X[i], c])
            h = self.rnn_cell(inp, h)
        output = self.fc(h).unsqueeze(0)
        return output

# Bidirectional RNN

In [ ]:
class BidirectionalRNN(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size, num_layers=1):
    super().__init__()
    self.rnn = nn.RNN(
          input_size=input_size,
          hidden_size=hidden_size,
          num_layers=num_layers,
          batch_first=True,
          bidirectional=True
        )
    self.fc = nn.Linear(2*hidden_size, vocab_size)

  def forward(self, x):
    output, h_last = self.rnn(x)
    logits = self.fc(output[:, -1, :])
    return logits

# Supporting methods

In [ ]:
def character_to_one_hot(c):
  onehot = torch.zeros(26)
  onehot[(ord(c) - ord('A'))] = 1
  return onehot

def seq_to_one_hot(seq):
  onehot = torch.zeros(len(seq), 26)
  for i, c in enumerate(seq):
    onehot[i] = character_to_one_hot(c)
  return onehot


In [ ]:
def train(model: nn.Module, dataset, epochs, lr = 1e-3, clip_grad = 1.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0.0

        for x_data, y_data in dataset:
            x = seq_to_one_hot(x_data).unsqueeze(0)
            y = torch.tensor([ord(y_data) - ord("A")])

            logits = model(x)
            loss = loss_fn(logits, y)

            optimizer.zero_grad()
            loss.backward()

            if clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataset)
        print(f"Epoch {epoch + 1}: Loss {avg_loss:.4f}")

def evaluate(model: nn.Module, dataset) -> float:
    correct = 0
    total = 0
    with torch.no_grad():
        for x_data, y_data in dataset:
            x = seq_to_one_hot(x_data).unsqueeze(0)
            y_idx = ord(y_data) - ord("A")

            logits = model(x)
            pred = torch.argmax(logits, dim=1).item()

            if pred == y_idx:
                correct += 1
            total += 1
    return 100.0 * correct / total

# Task 1


In [ ]:
train_loader, test_loader = generate_dataset(1000, task = 1)
print("Running the Shorter Task")

Running the Shorter Task


In [ ]:
print("\nRunning N-gram model with n=3")
ngram = Ngram(input_size=26, hidden_size=128, vocab_size=26, n=3)
train(ngram, train_loader, epochs = 10)
accuracy_ngram3 = evaluate(ngram, test_loader)
print("Trigram MLP accuracy: ", accuracy_ngram3, "%")

print("\nRunning RNN model")
rnn = RNN(input_size=26, hidden_size=128, vocab_size=26)
train(rnn, train_loader, epochs = 10)
accuracy_rnn = evaluate(rnn, test_loader)
print("RNN accuracy: ", accuracy_rnn,"%")

print("\nRunning LSTM model")
lstm = LSTM(input_size=26, hidden_size=128, vocab_size=26)
train(lstm, train_loader, epochs = 10)
accuracy_lstm = evaluate(lstm, test_loader)
print("LSTM accuracy: ", accuracy_lstm,"%")

print("\nRunning RNN w/ attention model")
rnn_attention = RNNAttention(input_size=26, hidden_size=128, attn_dim=64, vocab_size = 26)
train(rnn_attention, train_loader, epochs = 10)
accuracy_rnn = evaluate(rnn_attention, test_loader)
print("RNN w/ attention accuracy: ", accuracy_rnn,"%")

print("\nRunning Bidirectional RNN")
rnn_bidirectional = BidirectionalRNN(input_size=26, hidden_size=128, vocab_size = 26)
train(rnn_bidirectional, train_loader, epochs = 10)
accuracy_rnn_bi = evaluate(rnn_bidirectional, test_loader)
print("RNN w/ attention accuracy: ", accuracy_rnn_bi,"%")


Running N-gram model with n=3
Epoch 1: Loss 2.5022
Epoch 2: Loss 0.5985
Epoch 3: Loss 0.0725
Epoch 4: Loss 0.0203
Epoch 5: Loss 0.0101
Epoch 6: Loss 0.0061
Epoch 7: Loss 0.0040
Epoch 8: Loss 0.0028
Epoch 9: Loss 0.0020
Epoch 10: Loss 0.0015
Trigram MLP accuracy:  100.0 %

Running RNN model
Epoch 1: Loss 2.4103
Epoch 2: Loss 0.8771
Epoch 3: Loss 0.0812
Epoch 4: Loss 0.0083
Epoch 5: Loss 0.0037
Epoch 6: Loss 0.0021
Epoch 7: Loss 0.0014
Epoch 8: Loss 0.0009
Epoch 9: Loss 0.0007
Epoch 10: Loss 0.0005
RNN accuracy:  100.0 %

Running LSTM model
Epoch 1: Loss 2.9964
Epoch 2: Loss 2.4417
Epoch 3: Loss 2.0037
Epoch 4: Loss 1.3346
Epoch 5: Loss 0.6463
Epoch 6: Loss 0.2526
Epoch 7: Loss 0.1103
Epoch 8: Loss 0.0511
Epoch 9: Loss 0.0233
Epoch 10: Loss 0.0125
LSTM accuracy:  100.0 %

Running RNN w/ attention model
Epoch 1: Loss 2.5775
Epoch 2: Loss 1.6964
Epoch 3: Loss 0.7870
Epoch 4: Loss 0.1708
Epoch 5: Loss 0.0298
Epoch 6: Loss 0.0062
Epoch 7: Loss 0.0029
Epoch 8: Loss 0.0017
Epoch 9: Loss 0.001

# Task 2

In [ ]:
train_loader, test_loader = generate_dataset(5000, task = 2)
print("Running the Long Task")

Running the Long Task


In [ ]:
print("\nRunning N-gram model with n=3")
ngram = Ngram(input_size=26, hidden_size=128, vocab_size=26, n=3)
train(ngram, train_loader, epochs = 20)
accuracy_ngram3 = evaluate(ngram, test_loader)
print("Trigram MLP accuracy: ", accuracy_ngram3, "%")


Running N-gram model with n=3
Epoch 1: Loss 3.1721
Epoch 2: Loss 3.0540
Epoch 3: Loss 3.0415
Epoch 4: Loss 3.0368
Epoch 5: Loss 3.0342
Epoch 6: Loss 3.0325
Epoch 7: Loss 3.0312
Epoch 8: Loss 3.0301
Epoch 9: Loss 3.0290
Epoch 10: Loss 3.0280
Epoch 11: Loss 3.0269
Epoch 12: Loss 3.0258
Epoch 13: Loss 3.0244
Epoch 14: Loss 3.0229
Epoch 15: Loss 3.0211
Epoch 16: Loss 3.0191
Epoch 17: Loss 3.0168
Epoch 18: Loss 3.0143
Epoch 19: Loss 3.0115
Epoch 20: Loss 3.0090
Trigram MLP accuracy:  18.98101898101898 %


In [ ]:
print("\nRunning RNN model")
rnn = RNN(input_size=26, hidden_size=128, vocab_size=26)
train(rnn, train_loader, epochs = 20)
accuracy_rnn = evaluate(rnn, test_loader)
print("RNN accuracy: ", accuracy_rnn,"%")


Running RNN model
Epoch 1: Loss 3.2295
Epoch 2: Loss 3.0381
Epoch 3: Loss 2.9264
Epoch 4: Loss 2.8262
Epoch 5: Loss 2.7760
Epoch 6: Loss 2.7281
Epoch 7: Loss 2.6857
Epoch 8: Loss 2.7020
Epoch 9: Loss 2.6646
Epoch 10: Loss 2.6429
Epoch 11: Loss 2.6333
Epoch 12: Loss 2.6013
Epoch 13: Loss 2.5973
Epoch 14: Loss 2.6067
Epoch 15: Loss 2.5339
Epoch 16: Loss 2.5692
Epoch 17: Loss 2.5379
Epoch 18: Loss 2.5124
Epoch 19: Loss 2.4944
Epoch 20: Loss 2.4951
RNN accuracy:  29.47052947052947 %


In [ ]:
print("\nRunning RNN w/ attention model")
rnn_attention = RNNAttention(input_size=26, hidden_size=128, attn_dim=64, vocab_size = 26)
train(rnn_attention, train_loader, epochs = 10)
accuracy_rnn_att = evaluate(rnn_attention, test_loader)
print("RNN w/ attention accuracy: ", accuracy_rnn_att,"%")


Running RNN w/ attention model
Epoch 1: Loss 3.0914
Epoch 2: Loss 2.7174
Epoch 3: Loss 2.5607
Epoch 4: Loss 2.4420
Epoch 5: Loss 2.3520
Epoch 6: Loss 2.2481
Epoch 7: Loss 2.1375
Epoch 8: Loss 2.0818
Epoch 9: Loss 2.0586
Epoch 10: Loss 2.0984
RNN w/ attention accuracy:  23.076923076923077 %


In [ ]:
print("\nRunning LSTM model")
lstm = LSTM(input_size=26, hidden_size=128, vocab_size=26)
train(lstm, train_loader, epochs = 20)
accuracy_lstm = evaluate(lstm, test_loader)
print("LSTM accuracy: ", accuracy_lstm,"%")


Running LSTM model
Epoch 1: Loss 3.2325
Epoch 2: Loss 3.1068
Epoch 3: Loss 2.9693
Epoch 4: Loss 2.6998
Epoch 5: Loss 2.3271
Epoch 6: Loss 1.9817
Epoch 7: Loss 1.6979
Epoch 8: Loss 1.4300
Epoch 9: Loss 1.1703
Epoch 10: Loss 0.9368
Epoch 11: Loss 0.7345
Epoch 12: Loss 0.5590
Epoch 13: Loss 0.4328
Epoch 14: Loss 0.3440
Epoch 15: Loss 0.2502
Epoch 16: Loss 0.2262
Epoch 17: Loss 0.1979
Epoch 18: Loss 0.1693
Epoch 19: Loss 0.1724
Epoch 20: Loss 0.1547
LSTM accuracy:  37.46253746253746 %


In [ ]:
print("\nRunning Bidirectional RNN")
rnn_bi = BidirectionalRNN(input_size=26, hidden_size=128, vocab_size = 26)
train(rnn_bi, train_loader, epochs = 20)
accuracy_rnn_bi = evaluate(rnn_bi, test_loader)
print("RNN w/ attention accuracy: ", accuracy_rnn_bi,"%")


Running Bidirectional RNN
Epoch 1: Loss 3.2345
Epoch 2: Loss 3.0516
Epoch 3: Loss 2.8986
Epoch 4: Loss 2.8221
Epoch 5: Loss 2.7519
Epoch 6: Loss 2.7085
Epoch 7: Loss 2.6945
Epoch 8: Loss 2.6941
Epoch 9: Loss 2.6575
Epoch 10: Loss 2.6581
Epoch 11: Loss 2.6031
Epoch 12: Loss 2.5817
Epoch 13: Loss 2.5650
Epoch 14: Loss 2.5466
Epoch 15: Loss 2.5485
Epoch 16: Loss 2.5051
Epoch 17: Loss 2.5079
Epoch 18: Loss 2.4921
Epoch 19: Loss 2.4944
Epoch 20: Loss 2.4902
RNN w/ attention accuracy:  29.37062937062937 %
